In [3]:
import pandas as pd
import numpy as np
import os

# CSV 파일 경로 설정
input_file = "0. 조선_주식.csv"
output_file = "1. 조선_기술지표.csv"

# 1. CSV 파일 불러오기
df = pd.read_csv(input_file)

# 2. 날짜 형식 변환 및 2024년 데이터 필터링
df['날짜'] = pd.to_datetime(df['날짜'])
df = df[df['날짜'].dt.year == 2024].copy()

# 3. 기술지표 계산 함수 정의
def calculate_technical_indicators(group):
    group = group.sort_values('날짜').copy()

    # 이동평균
    group['MA5'] = group['종가'].rolling(window=5).mean()
    group['MA10'] = group['종가'].rolling(window=10).mean()

    # RSI 계산
    delta = group['종가'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    RS = gain / loss
    group['RSI'] = 100 - (100 / (1 + RS))

    # MACD 계산
    ema12 = group['종가'].ewm(span=12, adjust=False).mean()
    ema26 = group['종가'].ewm(span=26, adjust=False).mean()
    group['MACD'] = ema12 - ema26

    # 볼린저 밴드 상한선 (20일 MA + 2 * std)
    ma20 = group['종가'].rolling(window=20).mean()
    std20 = group['종가'].rolling(window=20).std()
    group['BB_upper'] = ma20 + (2 * std20)

    # 등락률 계산
    for days in [1, 3, 5]:
        group[f'{days}일후등락률'] = group['종가'].shift(-days)
        group[f'{days}일후등락률'] = (group[f'{days}일후등락률'] - group['종가']) / group['종가'] * 100

    return group

# 4. 기업별로 적용
df = df.groupby('기업명').apply(calculate_technical_indicators).reset_index(drop=True)

# 5. 필요한 컬럼만 선택 및 NaN 제거
columns = ['기업명', '날짜', '종가', '거래량', 'MA5', 'MA10', 'RSI', 'MACD', 'BB_upper', 
           '1일후등락률', '3일후등락률', '5일후등락률']
df = df[columns].dropna()

# 6. 결과 저장
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✅ '{output_file}' 파일이 생성되었습니다.")

✅ '1. 조선_기술지표.csv' 파일이 생성되었습니다.


C:\Users\human\AppData\Local\Temp\ipykernel_32544\3752164105.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('기업명').apply(calculate_technical_indicators).reset_index(drop=True)
